In [1]:
from src.processing.features.zyskowski import ZyskowskiFeatureExtractor
from src.processing.process import process
from src.model.data.dataset import DuelDataset

data = process(path="../../../res/json", extractor=ZyskowskiFeatureExtractor())
dataset = DuelDataset(features=data[0], labels=data[1])
print(dataset.x_temporal.shape)
print(dataset.x_snapshot.shape)

error parsing round: 'NoneType' object is not iterable
error parsing round: 'NoneType' object is not iterable
torch.Size([2690, 14, 4])
torch.Size([2690, 80])


In [2]:
import torch
from torch.utils.data import random_split

test_dataset, train_dataset = random_split(dataset, [0.2, 0.8], generator=torch.Generator().manual_seed(42))

In [4]:
print(len(dataset), len(test_dataset), len(train_dataset))

2690 538 2152


In [83]:
import os
from torch import optim, nn, utils, Tensor
import lightning as L


class Model(nn.Module):
    def __init__(self, temporal_size: int, snapshot_size: int):
        super().__init__()
        hidden_size = 128
        num_layers = 8
        self.lstm = nn.LSTM(temporal_size, hidden_size, dropout=0.3, bidirectional=False, batch_first=True,
                            num_layers=num_layers)
        self.tlinear = nn.Linear(hidden_size * num_layers, 32)
        self.slinear = nn.Linear(snapshot_size, 32)
        self.output_layer = nn.Linear(64, 1)

    def forward(self, temporal_x, snapshot_x):
        o, (h, c) = self.lstm(temporal_x)
        h = h.movedim(1, 0).flatten(start_dim=1)
        tfeatures = self.tlinear(h)
        sfeatures = self.slinear(snapshot_x)
        output_logit = self.output_layer(torch.cat([tfeatures, sfeatures], dim=-1))
        return output_logit


class LitModel(L.LightningModule):
    def __init__(self, ann):
        super().__init__()
        self.ann = ann

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x_temporal, x_snapshot, y = batch
        x_snapshot = x_snapshot.float()

        logit = self.ann(x_temporal, x_snapshot)
        loss = nn.functional.binary_cross_entropy(nn.functional.sigmoid(logit.flatten()), y.float())
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    # def validation_step(self, batch, batch_idx):
    #     # this is the test loop
    #     x_temporal, x_snapshot, y = batch
    #     logit = self.ann(x_temporal, x_snapshot)
    #     loss = nn.functional.binary_cross_entropy(nn.functional.sigmoid(logit.flatten()), y.float())
    #     self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [78]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=10)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=10)

for temporal, snapshot, y in train_dataloader:
    print(temporal.shape, snapshot.shape, y.shape)
    break

torch.Size([32, 14, 4]) torch.Size([32, 80]) torch.Size([32])


In [84]:
model = LitModel(Model(4, 80))
trainer = L.Trainer(max_epochs=50)
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type  | Params | Mode 
---------------------------------------
0 | ann  | Model | 1.0 M  | train
---------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.115     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Epoch 2:   0%|          | 0/68 [00:00<?, ?it/s, v_num=19, train_loss_step=37.50, train_loss_epoch=47.40]         

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x177a7ce50>
Traceback (most recent call last):
  File "/Users/ktz/opt/anaconda3/envs/cs2-duels/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/Users/ktz/opt/anaconda3/envs/cs2-duels/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1441, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/ktz/opt/anaconda3/envs/cs2-duels/lib/python3.10/multiprocessing/process.py", line 149, in join
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/ktz/opt/anaconda3/envs/cs2-duels/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/ktz/opt/anaconda3/envs/cs2-duels/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/ktz/opt/anaco

RuntimeError: DataLoader worker (pid(s) 43842) exited unexpectedly

In [68]:
model.ann

Model(
  (lstm): LSTM(4, 32, num_layers=4, batch_first=True, dropout=0.3)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [81]:
from torcheval.metrics import BinaryAccuracy

metric = BinaryAccuracy()
for temporal, snapshot, y in train_dataloader:
    y_hat = model.ann(temporal, snapshot.float()).flatten()
    # y_hat = torch.where(y_hat > 0.5, torch.ones_like(y_hat), torch.zeros_like(y_hat))
    metric.update(y_hat, y)
print("train", metric.compute())

metric = BinaryAccuracy()
for temporal, snapshot, y in test_dataloader:
    y_hat = model.ann(temporal, snapshot.float()).flatten()
    # y_hat = torch.where(y_hat > 0.5, torch.ones_like(y_hat), torch.zeros_like(y_hat))
    metric.update(y_hat, y)
print("test", metric.compute())

train tensor(0.6585)
test tensor(0.6115)
